In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
import pandas as pd

# Importing data

Getting data from kaggle first :

In [3]:
import pkg_resources

In [4]:
raw_data = pd.read_csv(pkg_resources.resource_stream('deepforest', 'data/train.csv'))

In [5]:
clean_data = raw_data.drop(["Cabin", "Name", "PassengerId", "Ticket"], axis=1)

In [6]:
clean_data = pd.get_dummies(clean_data).fillna(-1)

In [7]:
train, test = train_test_split(clean_data)

In [8]:
def split_x_y(dataframe, target):
    return dataframe.drop(target, axis=1), dataframe[target]

In [9]:
X_train, y_train = split_x_y(train, "Survived")
X_test, y_test = split_x_y(test, "Survived")

# Baseline model

In [10]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [11]:
y_pred = rf.predict_proba(X_test)

In [12]:
auc = roc_auc_score(y_true=y_test, y_score=y_pred[:, 1])

In [13]:
auc

0.86202702702702716

# Deep Forest

## By Hand

In [14]:
from sklearn.model_selection import StratifiedKFold

In [15]:
rf1 = RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=4)
rf2 = RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=10)

In [16]:
rf1.fit(X_train, y_train)
y_pred_train_1 = rf1.predict_proba(X_train)
y_pred_test_1 = rf1.predict_proba(X_test)

y_pred_train_1 = pd.DataFrame(y_pred_train_1, columns=["rf1_0", "rf1_1"], index=X_train.index)
y_pred_test_1 = pd.DataFrame(y_pred_test_1, columns=["rf1_0", "rf1_1"], index=X_test.index)

In [17]:
rf2.fit(X_train, y_train)
y_pred_train_2 = rf2.predict_proba(X_train)
y_pred_test_2 = rf2.predict_proba(X_test)

y_pred_train_2 = pd.DataFrame(y_pred_train_2, columns=["rf2_0", "rf2_1"], index=X_train.index)
y_pred_test_2 = pd.DataFrame(y_pred_test_2, columns=["rf2_0", "rf2_1"], index=X_test.index)

In [18]:
hidden_train_1 = pd.concat([X_train, y_pred_train_1, y_pred_train_2], axis=1)
hidden_test_1 = pd.concat([X_test, y_pred_test_1, y_pred_test_2], axis=1)

In [19]:
hidden_train_1.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,rf1_0,rf1_1,rf2_0,rf2_1
224,1,38.0,1,0,90.000,0,1,0,0,1,0.619984,0.380016,0.430290,0.569710
506,2,33.0,0,2,26.000,1,0,0,0,1,0.138802,0.861198,0.001357,0.998643
814,3,30.5,0,0,8.050,0,1,0,0,1,0.882710,0.117290,0.851545,0.148455
256,1,-1.0,0,0,79.200,1,0,1,0,0,0.059354,0.940646,0.020000,0.980000
392,3,28.0,2,0,7.925,0,1,0,0,1,0.890290,0.109710,0.906991,0.093009


In [20]:
rf3 = RandomForestClassifier(n_estimators=300, n_jobs=-1)
rf3.fit(hidden_train_1, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [21]:
y_pred3 = rf3.predict_proba(hidden_test_1)

In [22]:
roc_auc_score(y_test, y_pred3[:, 1])

0.85346846846846858

This is not very handy, not at all. We already see a lot of code duplication, and one may feel there may be a way to abstract a lot of the logic that is happening here, in a way that is more flexible and powerful that all this boilerplate code.

## With API

In [23]:
from deepforest.layer import Layer, InputLayer

In [24]:
input_layer = InputLayer(RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=4),
                         RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=10))

In [25]:
hidden_layer = Layer(input_layer,
                     RandomForestClassifier(n_estimators=50, n_jobs=-1, max_depth=4),
                     RandomForestClassifier(n_estimators=50, n_jobs=-1, max_depth=10))

In [26]:
hidden_layer.fit(X_train, y_train)

In [27]:
hidden_layer.predict(X_test).head()

,model0_0,model0_1,model1_0,model1_1
115,0.973218,0.026782,0.998333,0.001667
834,0.975804,0.024196,1.000000,0.000000
307,0.005117,0.994883,0.000000,1.000000
824,0.988266,0.011734,1.000000,0.000000
26,0.953786,0.046214,1.000000,0.000000


# Going Further

In [28]:
def random_forest_generator():
    for i in range(2, 15, 2):
        yield RandomForestClassifier(n_estimators=100,
                                     n_jobs=-1,
                                     min_samples_leaf=5,
                                     max_depth=i)
    for i in range(2, 15, 2):
        yield RandomForestClassifier(n_estimators=100,
                                     n_jobs=-1,
                                     max_features=1,
                                     min_samples_leaf=5,
                                     max_depth=i)

In [28]:
def paper_like_generator():
    for i in range(2):
        yield RandomForestClassifier(n_estimators=1000,
                                     n_jobs=-1,
                                     min_samples_leaf=10)
    for i in range(2):
        yield RandomForestClassifier(n_estimators=1000,
                                     n_jobs=-1,
                                     max_features=1,
                                     min_samples_leaf=10)

In [29]:
def build_input_layer():
    return InputLayer(*paper_like_generator())

In [30]:
def build_hidden_layer(layer):
    return Layer(layer, *paper_like_generator())

In [31]:
def build_output_layer(layer):
    return Layer(layer,
                 RandomForestClassifier(n_estimators=500,
                                        n_jobs=-1,
                                        min_samples_leaf=5,
                                        max_depth=10))

In [32]:
input_l = build_input_layer()
hidden_1 = build_hidden_layer(input_l)
hidden_2 = build_hidden_layer(hidden_1)
hidden_3 = build_hidden_layer(hidden_2)
hidden_4 = build_hidden_layer(hidden_3)
output_l = build_output_layer(hidden_4)

In [33]:
output_l.fit(X_train, y_train)

In [34]:
y_pred = output_l.predict(X_test)

In [36]:
y_pred

,model0_0,model0_1
115,1.000000,0.000000
834,1.000000,0.000000
307,0.000000,1.000000
824,1.000000,0.000000
26,1.000000,0.000000
230,0.000000,1.000000
733,1.000000,0.000000
792,1.000000,0.000000
680,0.000118,0.999882
283,1.000000,0.000000


In [35]:
roc_auc_score(y_test, y_pred.iloc[:, 1])

0.81846846846846855

Well the result is not that satisfactory yet, but let's not loose hope. There is a lot of room for improvement yet. First item on my todo list: make sure all the intermediary models are trained using cross-validation techniques, to reduce overfitting.